In [13]:
import os
import io
import time
import json
import datetime as dt
import typing
import requests

import pandas as pd
import snowflake.connector
from pypardot.client import PardotAPI, PardotAPIError
from xml.etree import ElementTree
import snowflake.connector


In [14]:
PARDOT_EMAIL="segment_integrations@discoveryed.com"
PARDOT_USER_KEY="f5a1dc61d079e35d9a2066a4a8498c32"
PARDOT_SF_CONSUMER_KEY="3MVG9IHf89I1t8hpom1l0QzbTANHH.MOASIZ3yPPhu6hFI_uipXiYi7ku85yMWg_2gXxu5bzAyWCevOHp4jrf"
PARDOT_SF_CONSUMER_SECRET="BC6C2AA5D41DC0EBBBB3376CD329E92E66BD8D85DF85F9FAB9F22E86D857A29E"
PARDOT_SF_REFRESH_TOKEN="5Aep8618yVsldz6rZPMv4ouelrGoRNAfdaLSjo3.ILw4jdNGemJVI_MNYFoNkN9g39GUXqUEgzw3ylEVMjxAU3X"
PARDOT_BUSINESS_UNIT_ID="0Uv4P000000TNB0SAO"
PARDOT_API_VERSION=4
PARDOT_MAX_RESULT_COUNT = 200
SNOWFLAKE_ACCOUNT_IDENTIFIER="UK29315.us-east-1"
SNOWFLAKE_USER="PardotEtlUser@discoveryed.com"
SNOWFLAKE_PASS="p@rd0tEtlUser"

In [15]:
SET_DATA_TYPE_CREATED = {
    # This set is for the data types that do not have a "updated_after" filter,
    # and must be queried usng the "created_after" filter
    "EmailClick",
    "TagObject",
}


In [16]:
def get_client_pardot() -> PardotAPI:
    try:
        p = PardotAPI(
            email=PARDOT_EMAIL,
            user_key=PARDOT_USER_KEY,
            sf_consumer_key=PARDOT_SF_CONSUMER_KEY,
            sf_consumer_secret=PARDOT_SF_CONSUMER_SECRET,
            sf_refresh_token=PARDOT_SF_REFRESH_TOKEN,
            business_unit_id=PARDOT_BUSINESS_UNIT_ID,
            version=PARDOT_API_VERSION,
        )

        if not p.authenticate():
            p.campaigns.query(limit=1)
            # ^ Must call query to authenticate if authenticatation is unsuccessful

    except PardotAPIError as err:
        if err.message in [
            "Error #184: access_token is invalid, unknown, or malformed",
            "Error #122: Daily API rate limit met",
        ]:
            get_session_boto().client("sns").publish(
                TargetArn="arn:aws:sns:us-east-1:768217030320:test-topic-email-notification",
                Message=f"PARDOT ERROR: {err.message}",
                # Message=json.dumps({"default": json.dumps(message)}),
                MessageStructure="string",
            )
        raise

    assert (
        p.sftoken != "dummy"
    ), "Pardot authentication still unsuccessful after querying"

    return p


In [17]:
p = get_client_pardot()

In [18]:
date_start = dt.date.today() - dt.timedelta(days=1)

In [19]:
data_type = 'TagObject'
DICT_CONVERSION_PURAL = {
    "Account": "accounts",
    "EmailClick": "emailclicks",
    "Email": "emails",
    "VisitorActivity": "visitoractivities",
    "Campaign": "campaigns",
    "List": "lists",
    "Prospect": "prospects",
    "TagObject": "tagobjects",
    "Tag": "tags",
    "Opportunity": "opportunities",
    "ProspectAccounts": "prospectaccounts",
    "Form": "forms",
    "ListMembership": "listmemberships",
    "Visitor": "visitors",
    "ProspectAccount": "prospectaccounts",
}
data_client = getattr(p, DICT_CONVERSION_PURAL[data_type])

In [20]:
hasattr(data_client, "query")

True

In [10]:
def helper_to_camelCase(s: str) -> str:
    "Return the same string with the first letter lowercase"
    return str(s[0].lower() + s[1:])


In [27]:
id_max = 0
data_raw = data_client.query(
                format="json",
                sort_by="id",
                id_greater_than=id_max,
                updated_after=date_start.isoformat(),
                **(
                    dict(created_after=date_start.isoformat())
                    if data_type in SET_DATA_TYPE_CREATED
                    else {}
                ),
            )

{'total_results': 62974,
 'tagObject': [{'id': 120097404,
   'tag_id': 1194704,
   'type': 'Prospect',
   'object_id': 52967034,
   'created_at': '2022-01-27 01:16:35'},
  {'id': 120097738,
   'tag_id': 1194704,
   'type': 'Prospect',
   'object_id': 23850486,
   'created_at': '2022-01-27 04:31:38'},
  {'id': 120097764,
   'tag_id': 1194704,
   'type': 'Prospect',
   'object_id': 18465613,
   'created_at': '2022-01-27 04:43:59'},
  {'id': 120099710,
   'tag_id': 1194704,
   'type': 'Prospect',
   'object_id': 52968166,
   'created_at': '2022-01-27 05:02:45'},
  {'id': 120100724,
   'tag_id': 1194704,
   'type': 'Prospect',
   'object_id': 39886489,
   'created_at': '2022-01-27 05:46:34'},
  {'id': 120100794,
   'tag_id': 1194292,
   'type': 'Email Draft',
   'object_id': 68832,
   'created_at': '2022-01-27 06:00:12'},
  {'id': 120100796,
   'tag_id': 1077864,
   'type': 'Email Draft',
   'object_id': 68832,
   'created_at': '2022-01-27 06:00:12'},
  {'id': 120100798,
   'tag_id': 33943

In [3]:
def get_client_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASS,
        account=SNOWFLAKE_ACCOUNT_IDENTIFIER,
        # authenticator="externalbrowser",
        warehouse="GENERAL_COMPUTE_WH",
        database="DEV_DATA_VAULT",
        schema="STAGE",
    )

In [12]:
ctx = get_client_snowflake()
cs = ctx.cursor()

#TODO, update when email table exists
snowflake_query = """select 
    a.list_email_id, min(a.email_id) 
    from dev_data_vault.marketing.visitor_activity a
    join (select distinct list_email_id from dev_data_vault.marketing.email) b
    on a.list_email_id = b.list_email_id
    where a.email_id is not null and a.list_email_id is not null 
    And b.list_email_id is null
    group by a.list_email_id
"""

snowflake_query_2 = """select 
    a.list_email_id, min(a.email_id)
    from dev_data_vault.marketing.visitor_activity a
    where a.email_id is not null and a.list_email_id is not null 
    group by a.list_email_id
"""

try:
    cs.execute(snowflake_query)
    missing_ids =  cs.fetchall()
except:
    cs.execute(snowflake_query_2)
    missing_ids =  cs.fetchall()   
finally:
    cs.close()
ctx.close()

missing